In [1]:
# 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력받는다
from selenium import webdriver as wb # 브라우저를 조작하는 도구
from selenium.webdriver.common.keys import Keys # 키 입력을 도와주는 도구(키보드)
from bs4 import BeautifulSoup as bs # 문서를 파싱해서 선택자 활용을 도와주는 도구
from tqdm import tqdm # 반복문 진행 정도를 시각화해주는 도구
from urllib.request import urlretrieve # 이미지 다운로드를 도와주는 도구
import time # 시간제어 도구
import os # 폴더 생성,삭제,이동 등을 도와주는 도구

### 병해충 도감 정보 검색

In [16]:
keyword = "가지"

# 크롬 드라이버를 사용하여 웹 브라우저를 실행한다.
path = 'c:/Users/user/알쓸농잡/chromedriver.exe'
driver = wb.Chrome(path)

driver.get('https://ncpms.rda.go.kr/npms/SicknsInfoListRM.np')
# 위의 페이지가 모두 열릴 때까지 2초 기다리기
time.sleep(2)

# 검색창의 이름을 찾아서 검색어를 입력
driver.find_element_by_id('sSearchWord').click()

element = driver.find_element_by_id('sSearchWord')
element.send_keys(keyword)

# 검색 버튼을 눌러 실행
driver.find_element_by_name('btn').click()

# 텍스트 추출하기
time.sleep(1)

full_html = driver.page_source

soup = bs(full_html, 'html.parser')

# content = soup.find('table', class_='grid')
content = driver.find_element_by_class_name('ce')

for i in content:
    print(i.text.strip())

TypeError: 'WebElement' object is not iterable

## 실습 문서 참고용

In [ ]:
keyword = "가지"

# 이미지가 저장될 폴더 생성
# 해당 폴더가 있는지 확인
if os.path.isdir('./{}'.format(keyword)) == False :
    os.mkdir('./{}'.format(keyword)) # 폴더 생성

In [ ]:
url = 'https://www.google.com/search?q={}&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjs85-GsN7vAhX4xYsBHR6aBg0Q_AUoAXoECAEQAw&biw=1745&bih=852'.format(keyword)

driver = wb.Chrome() # 브라우져 생성
driver.get(url) # url 요청
time.sleep(5) # 페이지 로딩까지 5초 대기

cnt = 0
pre_img_src = [] # 이전에 다운로드된 경로

for j in range(10) :
    img_html = bs(driver.page_source,'html.parser')

    # 이미지 태그 수집
    images = img_html.select('img.rg_i.Q4LuWd')

    # 이미지 태그의 src 속성 값 추출
    img_src = []
    for img in images :
        src = img.get('src')
        if src != None : # img 태그에 src 속성이 없는 경우
            if src not in pre_img_src : # 이전에 다운로드한 경로에 있는지 검사
                img_src.append(src)
        else : # img 태그에 src 속성이 있는 경우
            src = img.get('data-src')
            if src not in pre_img_src :
                img_src.append(src)

    # 파일 다운로드
    # img_src를 반복문으로 돌면서 저장, tqdm 사용
    for src in tqdm(img_src) :
        cnt += 1
        try :
            urlretrieve(src,'./{}/{}.png'.format(keyword,cnt))
        except :
            print("수집불가")
            continue

    pre_img_src += img_src # 다운로드한 경로를 이전 리스트에 추가    
        
    # 화면 스크롤
    for i in range(6):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)